# Importing libraries

In [61]:
# pandas
import pandas as pd


# plotting
import matplotlib.pyplot as plt

Dataset downloaded from kaggle link [here](https://www.kaggle.com/datasets/shrutipandit707/onlineretaildata)

Description of the dataset
-----------
This is an online retail dataset that can be used for frequency,recency and monetary analysis and for applying clustering algorithms such as K-Means. It's a great dataset for practicing Clustering Algorithms.

# Importing data and EDA

In [5]:
df = pd.read_csv('Online_Retail.csv')

In [10]:
# check few rows
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,1/12/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,1/12/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,1/12/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,1/12/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,1/12/2010 8:26,3.39,17850.0,United Kingdom


In [11]:
# data types of each features
df.dtypes

InvoiceNo       object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
UnitPrice      float64
CustomerID     float64
Country         object
dtype: object

## Duplicate row

In [30]:
100 * df[df.duplicated()].shape[0]/df.shape[0]

0.9721189350979592

0.97% training examples are duplicate row

* Question : Should the duplicate data be removed to run clustering algorithm? (Asked in [Stack](https://stats.stackexchange.com/questions/152808/do-i-need-to-remove-duplicate-objects-for-cluster-analysis-of-objects))

* Answer
    * "Having duplicates indicate that those are particularly likely combinations of variable values, which should get a higher weight because of that. This means observations with the same values do not become redundant."
    * If your data set has lots of duplicates, it can accelerate the processing a lot to merge them and use weights instead.

## Missing values

In [29]:
# which features has missing values() 
df.isna().any()

InvoiceNo      False
StockCode      False
Description     True
Quantity       False
InvoiceDate    False
UnitPrice      False
CustomerID      True
Country        False
dtype: bool

In [66]:
df["Description"].isna().sum()/df.shape[0], df["CustomerID"].isna().sum()/df.shape[0]

(0.002683107311375157, 0.249266943342886)

"Description"(0.27%) and "CustomerID" (24.9%) columns has missing entry

Question: What imputing strategy should we take for customerID?

## Encoding the categorical features

In [79]:

#df["CustomerID"].value_counts().sort_values(ascending=False).head(100).sum()/df.shape[0]

0.18132749225423456

In [68]:
df.shape

(541909, 8)